In [1]:
A_PDB = "../data/A.pdb"
data = None
with open(A_PDB, "r") as in_file:
	data = in_file.read()

In [2]:
def mb(string: str, k = 1024):
	return len(string) / (k*k)

In [3]:
# first I want to parse the string to read the unique characters in the file
def char_frequencies(string: str) -> dict[str, int]:
	freqs = {}
	for char in string:
		if char not in freqs:
			freqs[char] = 0
		else:
			freqs[char] += 1
	return freqs

In [4]:
# todo: make MODEL a character so I don't encode those separately and ATOM no need to model char separately
char_frequencies(data)

{'M': 27120,
 'O': 37836,
 'D': 3393,
 'E': 10295,
 'L': 14156,
 ' ': 887763,
 '1': 127074,
 '\n': 26754,
 'A': 64749,
 'T': 29876,
 'N': 11958,
 '-': 38823,
 '2': 95921,
 '5': 69215,
 '.': 133754,
 '8': 60099,
 '0': 111676,
 '4': 71186,
 '3': 76945,
 '9': 57091,
 '6': 64746,
 'C': 34267,
 '7': 62000,
 'B': 3201,
 'G': 8583,
 'S': 9036,
 'U': 4521,
 'H': 4195,
 'I': 2907,
 'P': 4253,
 'R': 6146,
 'V': 1462,
 'Z': 715,
 'Y': 5405}

In [5]:
freqs = char_frequencies(data)

In [6]:
class HuffNode():
	def __init__(self, freq: int, lchild: "HuffNode" = None, rchild: "HuffNode" = None):
		self.freq = freq
		self.lchild = lchild
		self.rchild = rchild
	def str(self):
		return f"({self.lchild.str()},{self.rchild.str()})"
	def __repr__(self):
		return self.__class__.__name__ + "(" + self.__dict__.__str__() + ")"

class HuffLeaf(HuffNode):
	def __init__(self, freq: int, char: str):
		super().__init__(freq)
		self.char = char
	def str(self):
		return f"'{self.char}'"

def is_leaf(node: HuffNode):
	return node.__class__ is HuffLeaf

a = HuffLeaf(1, 'A')
b = HuffNode(12)

print(is_leaf(a))
print(is_leaf(b))

True
False


In [7]:
from heapq import heapify, heappop, heappush

def node_to_heapq_format(node: HuffNode):
	return (node.freq, node) # high frequency ones should come up first


def heapq_format_to_node(heapq_item: tuple[int, HuffNode]):
	return heapq_item[1] # (priority, node)[1] selects node

class HuffQueue():
	def __init__(self, freqs: dict[str, int]):
		self.priority_queue = []
		heapify(self.priority_queue)
		
		# add all the leaves (characters) first
		for c, f in freqs.items():
			self.push(HuffLeaf(freq=f, char=c))

	def pop(self) -> HuffNode:
		return heapq_format_to_node(heappop(self.priority_queue))

	def push(self, new_node: HuffNode):
		heappush(self.priority_queue, node_to_heapq_format(new_node))
	
	def peak(self) -> HuffNode:
		return heapq_format_to_node(self.priority_queue[0])
	
	def __len__(self) -> int:
		return len(self.priority_queue)
	
	def __repr__(self) -> str:
		return f"HuffQueue(len={len(self)}, top={self.peak()})"
		
q = HuffQueue(freqs)

In [8]:
def create_huffman_code(freqs: dict[str, int]) -> HuffNode:
    """Returns the root node of the tree"""

    q = HuffQueue(freqs)
    while len(q) > 1:
        # pop two smallest nodes, they are now the bottom of the tree
        child_a, child_b = q.pop(), q.pop()
        parent = HuffNode(
            freq=child_a.freq + child_b.freq, 
            lchild=child_a, 
            rchild=child_b
        )
        # push the nodes (almost think of merged) back into circulation
        q.push(parent)

    return q.pop() # return the root


r = create_huffman_code(freqs)
r

HuffNode({'freq': 2167121, 'lchild': HuffLeaf({'freq': 887763, 'lchild': None, 'rchild': None, 'char': ' '}), 'rchild': HuffNode({'freq': 1279358, 'lchild': HuffNode({'freq': 523681, 'lchild': HuffNode({'freq': 255963, 'lchild': HuffLeaf({'freq': 127074, 'lchild': None, 'rchild': None, 'char': '1'}), 'rchild': HuffNode({'freq': 128889, 'lchild': HuffNode({'freq': 64143, 'lchild': HuffLeaf({'freq': 29876, 'lchild': None, 'rchild': None, 'char': 'T'}), 'rchild': HuffLeaf({'freq': 34267, 'lchild': None, 'rchild': None, 'char': 'C'})}), 'rchild': HuffLeaf({'freq': 64746, 'lchild': None, 'rchild': None, 'char': '6'})})}), 'rchild': HuffNode({'freq': 267718, 'lchild': HuffLeaf({'freq': 133754, 'lchild': None, 'rchild': None, 'char': '.'}), 'rchild': HuffNode({'freq': 133964, 'lchild': HuffLeaf({'freq': 64749, 'lchild': None, 'rchild': None, 'char': 'A'}), 'rchild': HuffLeaf({'freq': 69215, 'lchild': None, 'rchild': None, 'char': '5'})})})}), 'rchild': HuffNode({'freq': 755677, 'lchild': Huff

In [9]:
from treelib import Tree
def node_fmt(node: HuffNode):
	if is_leaf(node):
		if node.char == '\n':
			return f"""Leaf('\\n', "{node.freq:,}")"""
		else:
			return f"""Leaf('{node.char}', "{node.freq:,}")"""
	return f'Node("{node.freq:,}")'

def huffman_code_to_treelib(root: HuffNode):
	# add parent first
	global_id = 0
	t = Tree()
	t.create_node(node_fmt(root), global_id)

	# then add children
	def traverse(node: HuffNode, parent_id):
		nonlocal global_id, t
		if is_leaf(node):
			return

		global_id += 1
		lchild_id = global_id
		t.create_node(node_fmt(node.lchild), lchild_id, parent=parent_id)
		traverse(node.lchild, parent_id=lchild_id)
		global_id += 1
		rchild_id = global_id
		t.create_node(node_fmt(node.rchild), rchild_id, parent=parent_id)
		traverse(node.rchild, parent_id=rchild_id)



	traverse(root, global_id)

	return t

print(huffman_code_to_treelib(r))

Node("2,167,121")
├── Leaf(' ', "887,763")
└── Node("1,279,358")
    ├── Node("523,681")
    │   ├── Node("255,963")
    │   │   ├── Leaf('1', "127,074")
    │   │   └── Node("128,889")
    │   │       ├── Leaf('6', "64,746")
    │   │       └── Node("64,143")
    │   │           ├── Leaf('C', "34,267")
    │   │           └── Leaf('T', "29,876")
    │   └── Node("267,718")
    │       ├── Leaf('.', "133,754")
    │       └── Node("133,964")
    │           ├── Leaf('5', "69,215")
    │           └── Leaf('A', "64,749")
    └── Node("755,677")
        ├── Node("312,572")
        │   ├── Node("146,541")
        │   │   ├── Leaf('4', "71,186")
        │   │   └── Node("75,355")
        │   │       ├── Leaf('O', "37,836")
        │   │       └── Node("37,519")
        │   │           ├── Node("17,619")
        │   │           │   ├── Leaf('G', "8,583")
        │   │           │   └── Leaf('S', "9,036")
        │   │           └── Node("19,900")
        │   │               ├── Leaf('E', "1

In [10]:
str_repr = r.str()
print(str_repr)

(' ',((('1',(('T','C'),'6')),('.',('A','5'))),((('4',((('G','S'),(('U',(('Z','V'),'I')),'E')),'O')),('3',('-',((('Y','R'),'N'),'
')))),(('2','0'),((('M',('L',(('B','D'),('H','P')))),'9'),('8','7'))))))


In [11]:
def outer_comma(string: str):
    """"Returns None on failure and an int when found"""
    other = 0
    for i, char in enumerate(string):
        if other == 1 and char == ",":
            return i
        
        # if we encounter another paren, need to find its own matching pair
        if char == "(":
            other += 1
        elif char == ")":
            other -= 1

    return None
    

def parse_str_to_tree(string: str):
    # split a string at the upperlevel comma like "((a,b), c)" would split into "(a,b)" and "c"
    # i define the scope as anything inside the upper level parentheses
    # recursively do
    comma = outer_comma(string)
    a, b = string[:comma], string[comma+1:]
    b = b[:-1]
    a = a[1:]
    if a[0] != "'":
        a = parse_str_to_tree(a)
    else:
        a = a[1]
    if b[0] != "'":
        b = parse_str_to_tree(b)
    else:
        b = b[1]
    return a, b

def read_huffman_tree(tree_str: str) -> HuffNode:
    tuple_tree = parse_str_to_tree(tree_str)

    def construct(node):
        if isinstance(node, str):
            return HuffLeaf(freq=-1, char=node)
        return HuffNode(freq=-1, 
                 lchild=construct(node[0]),
                 rchild=construct(node[1]))

    return construct(tuple_tree)

read_huffman_tree("(('A','V'),('B',('EPIC','F')))")

HuffNode({'freq': -1, 'lchild': HuffNode({'freq': -1, 'lchild': HuffLeaf({'freq': -1, 'lchild': None, 'rchild': None, 'char': 'A'}), 'rchild': HuffLeaf({'freq': -1, 'lchild': None, 'rchild': None, 'char': 'V'})}), 'rchild': HuffNode({'freq': -1, 'lchild': HuffLeaf({'freq': -1, 'lchild': None, 'rchild': None, 'char': 'B'}), 'rchild': HuffNode({'freq': -1, 'lchild': HuffLeaf({'freq': -1, 'lchild': None, 'rchild': None, 'char': 'E'}), 'rchild': HuffLeaf({'freq': -1, 'lchild': None, 'rchild': None, 'char': 'F'})})})})

In [12]:
print(huffman_code_to_treelib(read_huffman_tree(str_repr)))

Node("-1")
├── Leaf(' ', "-1")
└── Node("-1")
    ├── Node("-1")
    │   ├── Node("-1")
    │   │   ├── Leaf('1', "-1")
    │   │   └── Node("-1")
    │   │       ├── Leaf('6', "-1")
    │   │       └── Node("-1")
    │   │           ├── Leaf('C', "-1")
    │   │           └── Leaf('T', "-1")
    │   └── Node("-1")
    │       ├── Leaf('.', "-1")
    │       └── Node("-1")
    │           ├── Leaf('5', "-1")
    │           └── Leaf('A', "-1")
    └── Node("-1")
        ├── Node("-1")
        │   ├── Node("-1")
        │   │   ├── Leaf('4', "-1")
        │   │   └── Node("-1")
        │   │       ├── Leaf('O', "-1")
        │   │       └── Node("-1")
        │   │           ├── Node("-1")
        │   │           │   ├── Leaf('G', "-1")
        │   │           │   └── Leaf('S', "-1")
        │   │           └── Node("-1")
        │   │               ├── Leaf('E', "-1")
        │   │               └── Node("-1")
        │   │                   ├── Leaf('U', "-1")
        │   │           

In [53]:
import math

def shallow_split(string: str, split_on=10):
    locs = []
    outer = 0
    for i in range(len(string)):
        char = string[i]
        if char == 40:
            outer += 1
        elif char == 41:
            outer -= 1
        
        if outer == 0 and char == split_on:
            locs.append(i)
    return locs

def leaves_to_encoding(code: HuffNode):
    # first find all the leaves
    # I can travel down, constructing a string, then when I hit, add that string to the global dict
    mapping = {}
    def trav(node: HuffNode, bits):
        if is_leaf(node):
            mapping[node.char] = bits
            return
        trav(node.lchild, bits+"0")
        trav(node.rchild, bits+"1")

    trav(code, "")
    return mapping


def encode_huff(code: HuffNode, original: str) -> str:
    # iterate over each character in the original, and convert it to bits form by 
    # considering a right turn as 1 and a left turn as 0 in the tree and when I hit a leaf stop
    # a conceptually easier way to think about this is starting from each leaf, going up and keeping track of how many it takes to go up to the top
    # and encoding based on which side it was, then creating a map that we can easily translate
    res = ""
    mapping = leaves_to_encoding(code)
    for char in original:
        res += mapping[char]
    return res

def decode_huff(root: HuffNode, encoded: str)  -> str:
    # iterate over the encoded bit string where left is 0 and right is 1
    # if I hit a leaf, then that is the character and restart
    cur_node = root
    result = ""
    for bit in encoded:
        if bit == "1":
            cur_node = cur_node.rchild
        elif bit == "0":
            cur_node = cur_node.lchild
        else: 
            print("ERROR")

        if is_leaf(cur_node):
            result += cur_node.char # this is the correct char
            cur_node = root # go back to the root node

    return result

def bits_str_to_bytes(bits: str):
    n = len(bits)
    byte = bits[:n]
    return int(byte, base=2).to_bytes(math.ceil(n/8))

def bytes_to_bits_str(b: bytes, actual_length):
    result = ""
    for chunk in b:
        result += f"{chunk:08b}"
    return result[len(result) - actual_length:]

class Huff:
    @staticmethod
    def encode(string: str) -> tuple[str, HuffNode]:
        freqs = char_frequencies(string)
        huffman_code = create_huffman_code(freqs)
        encoded = encode_huff(huffman_code, string)
        return encoded, huffman_code

    @staticmethod
    def decode(code: HuffNode, encoded: str) -> str:
        decoded = decode_huff(code, encoded)
        return decoded

def to_pz(pdb_file_name: str):
    """.pz means [p]rotein [z]ip"""
    data = None
    with open(pdb_file_name, "r") as pdb_file:
        data = pdb_file.read()

    encoded_str, huffman_code = Huff.encode(data)
    encoded = bits_str_to_bytes(encoded_str)

    NL = "\n".encode()
    with open("../null/example.pz", "wb") as pz_file:
        # two numbers, character length the location of the start of the tree
        num_chars = f"{len(data)}".encode() + NL

        huffman_tree_str = huffman_code.str().encode() + NL
        data_loc = f"{len(huffman_tree_str) + len(num_chars)}".encode() + NL

        # header
        pz_file.write(num_chars)
        pz_file.write(data_loc)
        # tree
        pz_file.write(huffman_tree_str)
        # data
        pz_file.write(encoded)

def from_pz(pz_filename="../null/example.pz"):
    with open(pz_filename, "rb") as pz_file:
        bs = pz_file.read()
        first_nl = bs.index(10)
        second_nl = bs[first_nl+1:].index(10) + first_nl

        chars = int(bs[:first_nl].decode())
        data_loc = int(bs[first_nl:second_nl+1])
        data_loc += len(str(data_loc)) + 1 # 1 for NL
        encoding = bs[data_loc:]
        tree_str = bs[second_nl+2:data_loc-1]

        encoding = bytes_to_bits_str(encoding, chars) 
        tree_str = tree_str.decode()
        huffman_code = read_huffman_tree(tree_str)
        decoded = Huff.decode(huffman_code, encoding)
        with open("../null/decoded.pdb", "w") as pdb_file:
            pdb_file.write(decoded)


        # then decode the text 

to_pz("../data/A.pdb")
from_pz()